## **section 1**

In [ ]:
 !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
 !tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
  .builder
  .appName("<app_name>")
  .getOrCreate())

In [ ]:
datasetPath="/content/drive/MyDrive/ipl_dataset_2/" 

In [ ]:
df_matches = spark.read.load(datasetPath+"ipl_matches.csv",
                     format="csv", sep=",", inferSchema=True, header=True)

In [ ]:
df_matches

DataFrame[match_id: int, date: string, player_of_match: string, venue_id: int, neutral_venue: int, team1: string, team2: string, toss_winner: string, toss_decision: string, winner: string, result: string, result_margin: string, eliminator: string, method: string, umpire1: string, umpire2: string]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_venue = spark.read.load(datasetPath+"ipl_venue.csv",
                     format="csv", sep=",", inferSchema=True, header=True)

In [ ]:
df_venue.head(5)

[Row(venue_id=1, venue='Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', city='Visakhapatnam'),
 Row(venue_id=2, venue='Sharjah Cricket Stadium', city='Sharjah'),
 Row(venue_id=3, venue='JSCA International Stadium Complex', city='Ranchi'),
 Row(venue_id=4, venue='Saurashtra Cricket Association Stadium', city='Rajkot'),
 Row(venue_id=5, venue='Shaheed Veer Narayan Singh International Stadium', city='Raipur')]

In [ ]:
df_ball_by_ball = spark.read.load(datasetPath+"ipl_ball_by_ball.csv",
                     format="csv", sep=",", inferSchema=True, header=True)

In [ ]:
df_ball_by_ball

DataFrame[match_id: int, inning: int, overs: int, ball: int, batsman: string, non_striker: string, bowler: string, batsman_runs: int, extra_runs: int, total_runs: int, non_boundary: int, is_wicket: int, dismissal_kind: string, player_dismissed: string, fielder: string, extras_type: string, batting_team: string, bowling_team: string]

In [ ]:
df_matches.createOrReplaceTempView("matches")

In [ ]:
df_venue.createOrReplaceTempView("venue")

In [ ]:
df_ball_by_ball.createOrReplaceTempView("ball_by_ball")

In [ ]:
spark.catalog.listTables()

[Table(name='ball_by_ball', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='matches', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='venue', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

## **section 2**

In [ ]:
m = df_matches.alias('m')
v= df_venue.alias('v')
b2b=df_ball_by_ball.alias('b2b')

In [ ]:
v.head()

Row(venue_id=1, venue='Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', city='Visakhapatnam')

In [ ]:
m.head()

Row(match_id=335982, date='2008-04-18', player_of_match='BB McCullum', venue_id=35, neutral_venue=0, team1='Royal Challengers Bangalore', team2='Kolkata Knight Riders', toss_winner='Royal Challengers Bangalore', toss_decision='field', winner='Kolkata Knight Riders', result='runs', result_margin='140', eliminator='N', method='NA', umpire1='Asad Rauf', umpire2='RE Koertzen')

In [ ]:
df_matches.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue_id: integer (nullable = true)
 |-- neutral_venue: integer (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



In [ ]:
df_ball_by_ball.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- overs: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- non_boundary: integer (nullable = true)
 |-- is_wicket: integer (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- fielder: string (nullable = true)
 |-- extras_type: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)



In [ ]:
df_venue.select('venue_id')

DataFrame[venue_id: int]

question 1


In [ ]:
from pyspark.sql.functions import col, desc

In [ ]:
EM=v.join(m, v.venue_id == m.venue_id)
EM2=EM.filter(EM.eliminator=='Y')
EM3=EM2.groupBy('venue').count()
EM3.sort(col("count").desc()).show(3)






+--------------------+-----+
|               venue|count|
+--------------------+-----+
|Dubai Internation...|    3|
|Sheikh Zayed Stadium|    2|
|Rajiv Gandhi Inte...|    1|
+--------------------+-----+
only showing top 3 rows



question 2


In [ ]:
players=b2b.filter(col("dismissal_kind")== 'caught').select('fielder')
player=players.groupBy('fielder').count()
player.sort(col("count").desc()).show(1)


+----------+-----+
|   fielder|count|
+----------+-----+
|KD Karthik|  118|
+----------+-----+
only showing top 1 row



question 3


In [ ]:
matches=m.filter(col("method")=='D/L').select('match_id')
bowler=b2b.filter((col("is_wicket")=='1') & ((b2b.dismissal_kind.isin('NA','runout'))==False)).select('bowler','match_id',).groupBy('bowler','match_id').count()
bowler.sort(col("count").desc()).show()






+-------------+--------+-----+
|       bowler|match_id|count|
+-------------+--------+-----+
|    DJG Sammy|  598056|    6|
|Sohail Tanvir|  336005|    6|
|   AD Russell|  980963|    6|
|      A Zampa|  980979|    6|
|    AS Joseph| 1178394|    6|
|     CV Varun| 1216497|    5|
|    VY Mahesh|  336032|    5|
|    MM Sharma|  729303|    5|
|     A Mishra|  336020|    5|
|      B Kumar| 1082609|    5|
|    RA Jadeja|  548311|    5|
|       AJ Tye| 1136600|    5|
|     A Kumble|  419115|    5|
|     A Kumble|  392182|    5|
|  JP Faulkner|  598032|    5|
|   JD Unadkat| 1082634|    5|
|    KV Sharma|  734037|    5|
|   SL Malinga|  501201|    5|
|   TG Southee| 1136599|    5|
|    SP Narine|  548323|    5|
+-------------+--------+-----+
only showing top 20 rows



question 4

In [ ]:
RR1=b2b.filter((b2b.overs>=7) & (b2b.extra_runs==0)) .select('batsman','batsman_runs').groupBy('batsman').sum('batsman_runs')
RR2=b2b.filter((b2b.overs>=7) & (b2b.extra_runs==0)) .select('batsman','ball').groupBy('batsman').count()
Join1=RR1.join(RR2, RR1.batsman==RR2.batsman).drop(RR1.batsman)
final=Join1.select('batsman',(((col("sum(batsman_runs)")/ (col("count")))*100).alias('strike_rate')))
final.sort(col("strike_rate").desc()).show(1)



+---------------+-----------+
|        batsman|strike_rate|
+---------------+-----------+
|KMDN Kulasekara|      250.0|
+---------------+-----------+
only showing top 1 row



question 5

In [ ]:
ER=b2b.filter(b2b.extra_runs!='0').select('extra_runs','match_id').groupBy('match_id').sum('extra_runs')
match=ER.join(m, ER.match_id==m.match_id)
venue=match.join(v, match.venue_id==v.venue_id)
ID=venue.select('sum(extra_runs)','v.venue_id')
ID.groupBy('venue_id').max('sum(extra_runs)')
final=ID.join(v, ID.venue_id==v.venue_id)
final2=final.select('venue','city','sum(extra_runs)')
final2.sort(col("sum(extra_runs)").desc()).show()



+--------------------+--------------+---------------+
|               venue|          city|sum(extra_runs)|
+--------------------+--------------+---------------+
|        Eden Gardens|       Kolkata|             38|
|   Brabourne Stadium|        Mumbai|             38|
|Himachal Pradesh ...|    Dharamsala|             36|
|M.Chinnaswamy Sta...|     Bengaluru|             36|
|   Brabourne Stadium|        Mumbai|             36|
|    Wankhede Stadium|        Mumbai|             35|
|MA Chidambaram St...|       Chennai|             35|
|    Wankhede Stadium|        Mumbai|             35|
|M.Chinnaswamy Sta...|     Bengaluru|             33|
|     SuperSport Park|     Centurion|             33|
|Sawai Mansingh St...|        Jaipur|             32|
|M.Chinnaswamy Sta...|     Bengaluru|             32|
|Sawai Mansingh St...|        Jaipur|             32|
|MA Chidambaram St...|       Chennai|             32|
|    Feroz Shah Kotla|         Delhi|             31|
|MA Chidambaram St...|      

question 6

In [ ]:
pom=m.filter(m.neutral_venue=='1').select('venue_id','player_of_match').groupBy('player_of_match').count()
pom.sort(col("count").desc()).show()


+----------------+-----+
| player_of_match|count|
+----------------+-----+
|       JP Duminy|    3|
|      GJ Maxwell|    3|
|       JH Kallis|    3|
|       YK Pathan|    3|
|  Sandeep Sharma|    2|
|    Yuvraj Singh|    2|
|       ML Hayden|    2|
|    SR Tendulkar|    2|
|  M Muralitharan|    2|
|       MK Pandey|    2|
|       RG Sharma|    2|
|DPMD Jayawardene|    2|
|        DR Smith|    2|
|   KC Sangakkara|    2|
|     LRPL Taylor|    2|
|        SK Raina|    2|
|  AB de Villiers|    2|
|    AC Gilchrist|    2|
|         M Vijay|    1|
|        V Sehwag|    1|
+----------------+-----+
only showing top 20 rows



question 7

In [ ]:
outs=b2b.filter(b2b.player_dismissed!='NA').select('player_dismissed').groupBy('player_dismissed').count()
runs=b2b.select('batsman','batsman_runs').groupBy('batsman').sum('batsman_runs')
average=runs.join(outs, outs.player_dismissed==runs.batsman)
final1=average.drop('player_dismissed')
final=final1.select('batsman',((col("sum(batsman_runs)"))/ (col("count"))).alias('average'))
final.sort(col("average").desc()).show()

+--------------+------------------+
|       batsman|           average|
+--------------+------------------+
|    MN van Wyk|55.666666666666664|
|    RD Gaikwad|              51.0|
|      AC Voges|             45.25|
|      KL Rahul| 44.86440677966102|
|       HM Amla| 44.38461538461539|
| Iqbal Abdulla|              44.0|
|     DA Warner| 42.71544715447155|
|   JM Bairstow|41.578947368421055|
|      CH Gayle| 41.13793103448276|
|      MS Dhoni|40.991150442477874|
|PD Collingwood|              40.6|
|AB de Villiers| 40.40833333333333|
|   LMP Simmons| 39.96296296296296|
|      SE Marsh| 39.95161290322581|
|     JP Duminy| 39.78431372549019|
| KS Williamson| 39.48780487804878|
|    MEK Hussey| 38.76470588235294|
|       V Kohli| 38.16883116883117|
|     ML Hayden|              36.9|
|     A Symonds|36.074074074074076|
+--------------+------------------+
only showing top 20 rows



question 8

In [ ]:
matches1=m.groupBy('umpire1').count()
matches1=matches1.withColumnRenamed("count","count1")
matches2=m.groupBy('umpire2').count()
count=matches1.join(matches2, matches1.umpire1 == matches2.umpire2)
count.drop('umpire2')
final=count.select('umpire1',((col("count1") + col("count")).alias("final count")))
final=final.withColumnRenamed("umpire1","umpire")
final.sort(col("final count").desc()).show(1)





+------+-----------+
|umpire|final count|
+------+-----------+
|S Ravi|        121|
+------+-----------+
only showing top 1 row



question 9

In [ ]:
vk=b2b.filter(b2b.batsman=='V Kohli').select('match_id','batsman_runs').groupBy('match_id').sum()
vk1=vk.drop('sum(match_id)')
vk2=vk1.withColumnRenamed("sum(batsman_runs)","runs")
join1=vk2.join(m, vk2.match_id == m.match_id)
join2=join1.join(v, join1.venue_id==v.venue_id)
final=join2.sort(col("runs").desc())
final.select('v.venue_id','venue','city').show(1)


+--------+--------------------+---------+
|venue_id|               venue|     city|
+--------+--------------------+---------+
|      35|M.Chinnaswamy Sta...|Bengaluru|
+--------+--------------------+---------+
only showing top 1 row



question 10

In [ ]:
winnert=m.filter(m.winner == m.toss_winner).select('winner').groupBy('winner').count().show()

table1=m.select('team1').groupBy('team1').count()
t1=table1.withColumnRenamed('count','matches1')
table2=m.select('team2').groupBy('team2').count()
t2=table2.withColumnRenamed('count','matches2')
join1=t1.join(t2,t1.team1==t2.team2).select('team1',((t1.matches1+t2.matches2).alias("matches"))).show()






+--------------------+-----+
|              winner|count|
+--------------------+-----+
| Sunrisers Hyderabad|   29|
| Chennai Super Kings|   61|
|Rising Pune Super...|    5|
|     Deccan Chargers|   19|
|Kochi Tuskers Kerala|    4|
|    Rajasthan Royals|   44|
|       Gujarat Lions|   10|
|Royal Challengers...|   43|
|Kolkata Knight Ri...|   55|
|Rising Pune Super...|    3|
|     Kings XI Punjab|   36|
|       Pune Warriors|    3|
|    Delhi Daredevils|   35|
|      Delhi Capitals|   10|
|      Mumbai Indians|   61|
+--------------------+-----+

+--------------------+-------+
|               team1|matches|
+--------------------+-------+
| Sunrisers Hyderabad|    124|
| Chennai Super Kings|    178|
|Rising Pune Super...|     16|
|     Deccan Chargers|     75|
|Kochi Tuskers Kerala|     14|
|    Rajasthan Royals|    161|
|       Gujarat Lions|     30|
|Royal Challengers...|    195|
|Kolkata Knight Ri...|    192|
|Rising Pune Super...|     14|
|     Kings XI Punjab|    190|
|       Pune W